## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.


In [ ]:
%pip install okareo 

**Example 1: Generate Scenario based on seed data passed as arguments**

In [2]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData


OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Generate example scenario based on seed data and return results in one call
scenario_set_create = ScenarioSetCreate(name="my test scenario set",
                                        number_examples=1,
                                        seed_data=[SeedData(input_="example question or statement", result="example result"),
                                                   SeedData(input_="tell me about your capability", result="example result"),
                                                   SeedData(input_="what are your limitations", result="example result")])

response = okareo.create_scenario_set(scenario_set_create)

print(response.scenario_input)


['Sample query or declaration', 'Can you inform me about your capacity?', 'can you tell me your constraints']


**Example 2: Create scenario set from a jsonl file upload with defined input and result fields**

In [3]:
import os
import tempfile
import json
from okareo import Okareo
from okareo.common import BASE_URL
from okareo_api_client.models import ScenarioSetCreate, ScenarioSetResponse, SeedData

# Step A - Create quick sample seed data file for example
# Sample seed data jsonl file for classification model, need input and result fields at minimum
rows = [
    {
        "input": "I have some quality concerns with your product, who can I talk to?",
        "result": "complaints"
    },
    {
        "input": "The product is not working as expected, I'd like to return it?",
        "result": "returns",

    },
    {
        "input": "I'd like to purchase additional filters for my model, how much are they?",
        "result": "pricing",
    }
]

temp_dir = tempfile.gettempdir()
file_path = os.path.join(temp_dir, "seed_data_sample.jsonl")

# Write to a .jsonl file
with open(file_path, "w+") as file:
    for row in rows:
        file.write(json.dumps(row) + '\n')
    

# Step B - Create scenario set with seed data file
OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

response = okareo.upload_scenario_set(file_path=file_path, scenario_name="Sample Scenario Upload")
print(response) # use scenario set id in output for the next step
source_scenario_id = response.scenario_id

# make sure to clean up tmp file
os.remove(file_path)

ScenarioSetResponse(scenario_id='ea85656b-78d4-4c71-beed-dd997816dd35', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 11, 0, 11, 15, 255396), type='SEED', tags=None, name='Sample Scenario Upload', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})


**Example 3: Generate a new Scenario Set by referring to another scenario set as seed data**

In [4]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioSetGenerate, ScenarioSetResponse


OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Use scenario set id from previous step as source for generation
response = okareo.generate_scenarios(source_scenario_id=source_scenario_id,
                                     name="generated scenario set",
                                     number_examples=2
                                     )

print(response)


ScenarioSetResponse(scenario_id='2475f292-7dc5-43d3-adb3-6a42fd223a60', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 11, 0, 11, 15, 291210), type='REPHRASE_INVARIANT', tags=['seed:ea85656b-78d4-4c71-beed-dd997816dd35'], name='generated scenario set', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})


**Example 4: Generate a new Scenario Set using the Text reverse question generator. This will generate a question for the scenario where the original input is the answer to the question generated**

In [3]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioType

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

response = okareo.upload_scenario_set(file_path='./webbizz_10_articles.jsonl', scenario_name="Retrieval Articles Scenario")
generated_scenario_response = okareo.generate_scenarios(
    source_scenario_id=response.scenario_id,
    name="questions from retrieval articles",
    number_examples=2,
    generation_type=ScenarioType.TEXT_REVERSE_QUESTION
)

print(generated_scenario_response)

ScenarioSetResponse(scenario_id='e86f9279-2727-4f57-a2ac-87e7997534c2', project_id='f7bfcff0-2b8d-4fec-ab20-51669cd3e732', time_created=datetime.datetime(2024, 1, 17, 19, 37, 39, 196001), type='TEXT_REVERSE_QUESTION', tags=['seed:f383bffa-be81-48d2-a6c9-7dc570dcee42'], name='questions from retrieval articles', seed_data=[], scenario_count=0, scenario_input=[], additional_properties={})
